# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234498952336                   -0.50    8.0
  2   -7.250209056781       -1.80       -1.40    1.0
  3   -7.251247385283       -2.98       -2.04    4.0
  4   -7.251082788862   +   -3.78       -1.96    4.0
  5   -7.251332558002       -3.60       -2.74    4.0
  6   -7.251338343242       -5.24       -3.24    3.0
  7   -7.251338795433       -6.34       -4.21    3.0
  8   -7.251338797763       -8.63       -4.54    4.0
  9   -7.251338798631       -9.06       -5.14    2.0
 10   -7.251338798703      -10.14       -5.85    3.0
 11   -7.251338798704      -11.93       -6.36    4.0
 12   -7.251338798704      -13.21       -6.55    4.0
 13   -7.251338798705      -12.94       -7.36    3.0
 14   -7.251338798705      -15.05       -8.15    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.040972475905489414
[ Info: Arnoldi iteration step 2: normres = 0.48201410238910003
[ Info: Arnoldi iteration step 3: normres = 0.34225749286981344
[ Info: Arnoldi iteration step 4: normres = 1.0516154207732127
[ Info: Arnoldi iteration step 5: normres = 0.21043958921604222
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.22e-01, 2.85e-02, 1.08e-01, 1.18e-01, 5.55e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5467193467099244
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.89e-02, 1.87e-02, 5.24e-01, 1.18e-01, 5.02e-02)
[ Info: Arnoldi iteration step 7: normres = 0.11393491731690937
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.26e-03, 9.25e-03, 8.21e-02, 3.04e-02, 6.24e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10311066413002098
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.4